In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
fem_act   = pd.read_excel('data/Mouse_Data.xlsx', sheet_name='Fem Act')   # contains a “Minute” column + F1,F2…
male_act  = pd.read_excel('data/Mouse_Data.xlsx', sheet_name='Male Act')
fem_temp  = pd.read_excel('data/Mouse_Data.xlsx', sheet_name='Fem Temp')
male_temp = pd.read_excel('data/Mouse_Data.xlsx', sheet_name='Male Temp')

fem_act   = fem_act.iloc[:, 1:].copy()
male_act  = male_act.iloc[:, 1:].copy()
fem_temp  = fem_temp.iloc[:, 1:].copy()
male_temp = male_temp.iloc[:, 1:].copy()



In [3]:
N = fem_act.shape[0]
assert all(df.shape[0] == N for df in (male_act, fem_temp, male_temp))

In [4]:
minutes = pd.RangeIndex(start=0, stop=N, name='idx')
meta = pd.DataFrame({
    'day':           (minutes // 1440) + 1,
    'minute_of_day': minutes % 1440
}, index=minutes)

estrus_days = [2, 6, 10, 14]
is_estrus   = meta['day'].isin(estrus_days)

female_ids = fem_act.columns.tolist()
male_ids   = male_act.columns.tolist()

In [5]:
plt.figure(figsize=(8,4))
mean_f = fem_act.groupby(meta['minute_of_day']).mean().mean(axis=1)
mean_m = male_act.groupby(meta['minute_of_day']).mean().mean(axis=1)
plt.plot(mean_f.index, mean_f.values, label='Female')
plt.plot(mean_m.index, mean_m.values, label='Male')
plt.xlabel('Minute of Day')
plt.ylabel('Mean Activity')
plt.title('Average 24-h Activity Profile by Sex')
plt.legend()
plt.tight_layout()
plt.savefig('fig1_mean_activity_by_sex.png')
plt.close()

In [6]:
plt.figure(figsize=(8,4))
# Female grouped by minute & estrus
ft_grp = fem_temp.groupby([meta['minute_of_day'], is_estrus]).mean()
mean_non = ft_grp.xs(False, level=1).mean(axis=1)
mean_yes = ft_grp.xs(True,  level=1).mean(axis=1)
# Male overall
mean_male_temp = male_temp.groupby(meta['minute_of_day']).mean().mean(axis=1)

plt.plot(mean_non.index, mean_non.values, '-',  label='Female Non-Estrus')
plt.plot(mean_yes.index, mean_yes.values, '--', label='Female Estrus')
plt.plot(mean_male_temp.index, mean_male_temp.values, ':', label='Male (all days)')
plt.xlabel('Minute of Day')
plt.ylabel('Core Temperature (°C)')
plt.title('24-h Temperature Rhythms: Females & Males')
plt.legend()
plt.tight_layout()
plt.savefig('fig2_temperature_rhythm.png')
plt.close()

In [7]:
rep_f = female_ids[0]
rep_m = male_ids[0]
mat_f = fem_act[rep_f].values.reshape(14, 1440)
mat_m = male_act[rep_m].values.reshape(14, 1440)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,6), sharex=True)
ax1.imshow(mat_f, aspect='auto', origin='lower')
ax1.set_title(f'Female {rep_f} Activity Heatmap')
ax1.set_ylabel('Day')
ax2.imshow(mat_m, aspect='auto', origin='lower')
ax2.set_title(f'Male {rep_m} Activity Heatmap')
ax2.set_ylabel('Day')
ax2.set_xlabel('Minute of Day')
plt.tight_layout()
plt.savefig('fig3_activity_heatmaps.png')
plt.close()

In [8]:
df_t = fem_temp.copy()
df_t['day']           = meta['day']
df_t['minute_of_day'] = meta['minute_of_day']
df_t['is_estrus']     = is_estrus

# Keep only dark phase (minutes ≥ 720)
dark_df = df_t[df_t['minute_of_day'] >= 720]
# Mean temp per day & estrus flag (one value per mouse per day)
grp = dark_df.groupby(['day','is_estrus'])[female_ids].mean()
# Flatten to two arrays
vals_non = grp.xs(False, level=1).values.flatten()
vals_yes = grp.xs(True,  level=1).values.flatten()

plt.figure(figsize=(6,4))
plt.boxplot([vals_non, vals_yes], labels=['Non-Estrus','Estrus'])
plt.ylabel('Mean Dark-Phase Temp (°C)')
plt.title('Dark-Phase Temperature by Estrus Status')
plt.tight_layout()
plt.savefig('fig4_boxplot_temp.png')
plt.close()

/var/folders/sd/54269wrn4fz6bkfc0cnb4n4h0000gn/T/ipykernel_5018/75177944.py:15: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot([vals_non, vals_yes], labels=['Non-Estrus','Estrus'])


In [9]:
rng = np.random.default_rng(0)
idx_sample = rng.choice(N, size=20000, replace=False)
x = fem_act.iloc[idx_sample][rep_f]
y = fem_temp.iloc[idx_sample][rep_f]
c = is_estrus.iloc[idx_sample].astype(int)

plt.figure(figsize=(6,6))
plt.scatter(x, y, s=2, alpha=0.5, c=c)
plt.xlabel('Activity')
plt.ylabel('Core Temperature (°C)')
plt.title(f'Activity vs Temp for {rep_f} (colored by estrus)')
plt.tight_layout()
plt.savefig('fig5_scatter_activity_temp.png')
plt.close()

In [10]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,6), sharex=True)
ax1.plot(fem_act.index, fem_act[rep_f], linewidth=0.5)
ax1.set_ylabel('Activity')
ax1.set_title(f'{rep_f} Activity Over 14 Days')
ax2.plot(fem_temp.index, fem_temp[rep_f], linewidth=0.5)
ax2.set_ylabel('Temperature (°C)')
ax2.set_title(f'{rep_f} Temperature Over 14 Days')

for d in estrus_days:
    onset = (d - 1) * 1440
    ax1.axvline(onset, linestyle='--', linewidth=1)
    ax2.axvline(onset, linestyle='--', linewidth=1)

ax2.set_xlabel('Minute Index')
plt.tight_layout()
plt.savefig('fig6_timeseries_estrus.png')
plt.close()